Надо валидировать, посмотреть на конкретные примеры и попробовать интерпретировать попадания и ошибки.

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/NIR/Data/submission_complete.csv')
data.head(3)

,Unnamed: 0,id,year,year_id,title,submission,track,author keywords,easychair keyphrases,topics,...,submitted,last update,address,submission group,decision,attachment,paper,publication,presentation preference,file_name
0,0,5,2014,2014_5,Information System For Integrated Monitoring A...,https://easychair.org/conferences/submission_d...,Main Track,Incremental and iterative\nDecision support sy...,"public health faculty university (160), social...",Modelling and simulation in the medical scienc...,...,"Sep 19, 17:38 GMT","Sep 19, 17:38 GMT",Biostatistics and Health Informatics Departmen...,NaN,OUT OF SCOPE,NaN,NaN,NaN,NaN,ICCS_2014_submission_5.pdf
1,1,7,2014,2014_7,Analyzing buffer contents of Space Division Ou...,https://easychair.org/conferences/submission_d...,Main Track,Data Communications\nPerformance Analysis\nQue...,"system occupancy (200), service rate (100), ge...",Modelling and simulation of complex systems,...,"Sep 26, 06:52 GMT","Sep 26, 06:52 GMT","Department of Information Technology, College ...",NaN,OUT OF SCOPE,NaN,NaN,NaN,NaN,ICCS_2014_submission_7.pdf
2,2,10,2014,2014_10,A new kind of complexity,https://easychair.org/conferences/submission_d...,Main Track,MSC classes: 11Y16\nACM classes: G.0\ncomputat...,"random oracle (160), parity sequence (150), co...","Unconventional computing, bio-inspired and nat...",...,"Nov 05, 21:37 GMT","Nov 05, 21:37 GMT","16 Hepworth Street\nArundel, 4214\nAustralia",NaN,OUT OF SCOPE,NaN,NaN,NaN,NaN,ICCS_2014_submission_10.pdf


In [ ]:
data = data[['abstract','decision']]
data=data[data['decision'].isin(['REJECT','ACCEPT','OUT OF SCOPE', 'ACCEPT - Full Paper', 'ACCEPT - Short Paper'])]
data['decision'].value_counts()

decision
REJECT                  775
ACCEPT                  492
ACCEPT - Full Paper     158
OUT OF SCOPE             92
ACCEPT - Short Paper     87
Name: count, dtype: int64

In [ ]:
mapper = {'ACCEPT':'REJECT', 'ACCEPT - Full Paper':'OUT OF SCOPE', 'ACCEPT - Short Paper':'OUT OF SCOPE', 'REJECT':'REJECT', 'OUT OF SCOPE':'OUT OF SCOPE'}
data['decision'] = data['decision'].map(mapper)
data['decision'].value_counts()

decision
REJECT          1267
OUT OF SCOPE     337
Name: count, dtype: int64

In [ ]:
data.head()

,abstract,decision
0,Indonesian government delivered new program fo...,OUT OF SCOPE
1,"In digital communication systems, buffers are ...",OUT OF SCOPE
2,A new class of functions is presented. The str...,OUT OF SCOPE
3,Computing a hierarchical clustering of objects...,REJECT
4,Steady state polymerization of propylene using...,REJECT


In [ ]:
print(round(data['decision'][data['decision']=='REJECT'].count()/1604*100,2))
print(round(data['decision'][data['decision']=='OUT OF SCOPE'].count()/1604*100,2))

78.99
21.01


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from string import punctuation
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
morph = MorphAnalyzer()

def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in punctuation])
    #tokens = text.split()
    text = ' '.join([word for word in text.split() if word not in stop_words])
    #tokens = [token for token in tokens if token not in stop_words]
    text = ' '.join([morph.parse(word)[0].normal_form for word in text.split()])
    #tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return text

data['abstract'] = data['abstract'].apply(preprocess_text)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=130b8384872dbd7cd395f3b560bbf39617c745b47db1e292e5de76ff0ab03f13
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data['abstract']

0       indonesian government delivered new program so...
1       digital communication systems buffers used tem...
2       new class functions presented structure algori...
3       computing hierarchical clustering objects pair...
4       steady state polymerization propylene using sp...
                              ...                        
1796    n contribution simple efficient algorithm clos...
1797    logistics key component supply chain firm amon...
1799    complex particle reconstruction software used ...
1800    rapid influx fake news nowadays poses immense ...
1802    resource allocation mitigation recovery activi...
Name: abstract, Length: 1604, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=2, max_df=0.5)

X_tfidf = tfidf_vectorizer.fit_transform(data['abstract'])

feature_names_tfidf = tfidf_vectorizer.get_feature_names_out()

print("Словарь TF-IDF:")
print(feature_names_tfidf[:20])
print("\nМатрица TF-IDF признаков:")
print(X_tfidf.toarray())

Словарь TF-IDF:
['01' '02' '03' '08' '087' '10' '10 percent' '10 total' '10 years' '100'
 '1000' '1024' '105' '109' '10fold' '10fold crossvalidation' '11' '12'
 '126x' '128']

Матрица TF-IDF признаков:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
!pip install scikit-learn imbalanced-learn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

X = data["abstract"]
y = data["decision"]

vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_encoded, test_size=0.2, random_state=42)

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

class_counts = pd.Series(y_train_resampled).value_counts()
scale_pos_weight = class_counts[0] / class_counts[1]

model = xgb.XGBClassifier(scale_pos_weight=scale_pos_weight, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train_resampled, y_train_resampled)

y_pred = model.predict(X_test)

y_test_labels = label_encoder.inverse_transform(y_test)
y_pred_labels = label_encoder.inverse_transform(y_pred)

print("Classification Report:")
print(classification_report(y_test_labels, y_pred_labels))
print("Confusion Matrix:")
print(confusion_matrix(y_test_labels, y_pred_labels))

Classification Report:
              precision    recall  f1-score   support

OUT OF SCOPE       0.33      0.18      0.23        61
      REJECT       0.83      0.92      0.87       260

    accuracy                           0.78       321
   macro avg       0.58      0.55      0.55       321
weighted avg       0.73      0.78      0.75       321

Confusion Matrix:
[[ 11  50]
 [ 22 238]]


In [ ]:
# Создание DataFrame для анализа ошибок
results_df = pd.DataFrame({
    'abstract': X.iloc[y_test].values,
    'true_label': y_test_labels,
    'predicted_label': y_pred_labels
})

# Вывод примеров неправильных предсказаний
incorrect_predictions = results_df[results_df['true_label'] != results_df['predicted_label']]
print("\nIncorrect Predictions:")
print(incorrect_predictions.head(10))

# Вывод примеров правильных предсказаний
correct_predictions = results_df[results_df['true_label'] == results_df['predicted_label']]
print("\nCorrect Predictions:")
print(correct_predictions.head(10))


Incorrect Predictions:
                                             abstract    true_label  \
12  digital communication systems buffers used tem...        REJECT   
17  indonesian government delivered new program so...  OUT OF SCOPE   
23  indonesian government delivered new program so...  OUT OF SCOPE   
24  digital communication systems buffers used tem...        REJECT   
29  indonesian government delivered new program so...  OUT OF SCOPE   
32  indonesian government delivered new program so...  OUT OF SCOPE   
33  indonesian government delivered new program so...  OUT OF SCOPE   
35  indonesian government delivered new program so...  OUT OF SCOPE   
40  digital communication systems buffers used tem...        REJECT   
45  indonesian government delivered new program so...  OUT OF SCOPE   

   predicted_label  
12    OUT OF SCOPE  
17          REJECT  
23          REJECT  
24    OUT OF SCOPE  
29          REJECT  
32          REJECT  
33          REJECT  
35          REJECT  
40   